## Multiclass classification - iris
- 이번에는 0, 1 분류가 아닌 multiclass분류
- 먼저 간단한 iris data를 이용

## 1. Import Required Libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import random

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

from torch.utils.data import Dataset
from torch.utils.data import DataLoader


## 2. Generate Data

In [6]:
# iris data를 불러옵니다. 

iris = load_iris()
print(iris.data.shape)    # 150개의 data 4개의 feature
print(iris.target.shape)  

(150, 4)
(150,)


In [9]:
# train과 test set을 분리합니다.

train_X, test_X, train_y, test_y = train_test_split(iris.data, iris.target, test_size=0.3)

In [24]:
test_X

array([[5.5, 3.5, 1.3, 0.2],
       [5.7, 2.6, 3.5, 1. ],
       [4.9, 3. , 1.4, 0.2],
       [6.7, 3.1, 4.7, 1.5],
       [5.3, 3.7, 1.5, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [5.6, 2.7, 4.2, 1.3],
       [5. , 2. , 3.5, 1. ],
       [5.1, 2.5, 3. , 1.1],
       [6.8, 2.8, 4.8, 1.4],
       [6. , 2.2, 4. , 1. ],
       [7.2, 3.6, 6.1, 2.5],
       [6.7, 3.3, 5.7, 2.1],
       [5.1, 3.8, 1.9, 0.4],
       [6.3, 2.5, 5. , 1.9],
       [6.1, 2.8, 4. , 1.3],
       [5.9, 3.2, 4.8, 1.8],
       [6.4, 2.8, 5.6, 2.1],
       [4.8, 3.1, 1.6, 0.2],
       [6. , 2.7, 5.1, 1.6],
       [6.3, 3.3, 6. , 2.5],
       [4.9, 2.4, 3.3, 1. ],
       [7.2, 3. , 5.8, 1.6],
       [4.9, 3.1, 1.5, 0.1],
       [5. , 3.5, 1.6, 0.6],
       [7.3, 2.9, 6.3, 1.8],
       [5.8, 2.7, 4.1, 1. ],
       [4.6, 3.4, 1.4, 0.3],
       [6.3, 3.4, 5.6, 2.4],
       [5.6, 2.8, 4.9, 2. ],
       [5.1, 3.8, 1.5, 0.3],
       [5.2, 3.4, 1.4, 0.2],
       [6.3, 2.3, 4.4, 1.3],
       [6.4, 2.9, 4.3, 1.3],
       [5.1, 3

In [13]:
class makeData(Dataset):
    def __init__(self,X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self,index):
        return self.X_data[index], self.y_data[index]
    
    def __len__(self):
        return len(self.X_data)

In [14]:
# normalize는 기본적으로 열(column)기준입니다
train_data = makeData(np.array(normalize(train_X)), np.array(train_y))
test_data = makeData(np.array(normalize(test_X)), np.array(test_y))

In [15]:
train_data.__getitem__(0)

(array([0.76701103, 0.35063361, 0.51499312, 0.15340221]), 1)

In [16]:
train_data.__len__()

105

## 3. Model & Optimizer

In [28]:
class Multiclass_Classification(nn.Module):
    def __init__(self,num_feature, num_class):
        super(Multiclass_Classification, self).__init__()
        
        # 이전 예제에서는 __init__에 self만 있었습니다.
        # 이제는 넣고 싶은 feature의 개수와 class를 자유롭게 하기 위해 변수로 지정합니다.
        # Layer 역시 활성화 함수를 ReLU로 변경하고 Layer안에 넣어 좀 더 짧은 코드로 신경망을 구성해보았습니다.
        
        self.Layer_1 = nn.Sequential(
                            nn.Linear(num_feature,100),
                            nn.ReLU()
                        )
        
        self.Layer_2 = nn.Sequential(
                            nn.Linear(100,50),
                            nn.ReLU()
                        )
        
        self.Layer_3 = nn.Sequential(
                            nn.Linear(50,30),
                            nn.ReLU()
                        )
        
        self.dropout = nn.Dropout(0.5)
        
        self.out = nn.Linear(30, num_class)
        
    def forward(self, inputs):
        
        x = self.Layer_1(inputs)
        x = self.Layer_2(x)
        x = self.dropout(x)
        x = self.Layer_3(x)
        x = self.dropout(x)
        
        x = self.out(x)
        
        return x
    
    def predict(self, test_inputs):
        x = self.Layer_1(test_inputs)
        x = self.Layer_2(x)
        x = self.Layer_3(x)
        
        x = self.out(x)
        
        # 1차원의 최대값 위치들을 출력합니다
        
        return torch.max(x,1)[1]

## 4. Training

In [29]:
## 30개씩 1000번 학습시켜보겠습니다
EPOCHS = 1000
BATCH_SIZE = 30
NUM_FEATURE = 4
NUM_CLASS = 3

model = Multiclass_Classification(NUM_FEATURE, NUM_CLASS)

# pytorch 에서는 NLL loss + log softmax가 이미 적용되어있습니다. 
criterion = nn.CrossEntropyLoss()

train_loader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)

optimizer = optim.Adam(model.parameters(), lr = 0.01)

for epoch in range(EPOCHS):
    
    for X_batch, y_batch in train_loader:
        
        # torch.tensor는 기본적으로 float로 들어갑니다.
        
        inputs = torch.Tensor(X_batch.float())
        targets = torch.LongTensor(y_batch.long()) # dummy로 들어가나?
        model.zero_grad()
        y_pred = model(inputs)
        loss = criterion(y_pred, targets)
        loss.backward()
        optimizer.step()
        
    if epoch % 100 == 0:
        print(loss)
        

tensor(1.1304, grad_fn=<NllLossBackward>)
tensor(0.1356, grad_fn=<NllLossBackward>)
tensor(0.0882, grad_fn=<NllLossBackward>)
tensor(0.0079, grad_fn=<NllLossBackward>)
tensor(0.1193, grad_fn=<NllLossBackward>)
tensor(0.0584, grad_fn=<NllLossBackward>)
tensor(0.1681, grad_fn=<NllLossBackward>)
tensor(0.0190, grad_fn=<NllLossBackward>)
tensor(0.0995, grad_fn=<NllLossBackward>)
tensor(0.0501, grad_fn=<NllLossBackward>)


## 5. Check Trained Parameters

In [34]:
test_loader = DataLoader(dataset = test_data, batch_size = 1)

test_y_true = []
test_y_pred = []

for X_batch, y_batch in test_loader:
    inputs = torch.Tensor(X_batch.float())
    test_y_true.append(y_batch.item())
    y_pred = model.predict(inputs)
    test_y_pred.append(int(y_pred.item()))

In [36]:
print(test_y_pred)

[0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 2, 2, 0, 2, 1, 2, 2, 0, 2, 2, 1, 1, 0, 0, 2, 1, 0, 2, 2, 0, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 1, 1, 2]


In [37]:
print(test_y_true)

[0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 2, 2, 0, 2, 1, 1, 2, 0, 1, 2, 1, 2, 0, 0, 2, 1, 0, 2, 2, 0, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 1, 1, 2]


In [ ]:
metrics.accuracy_score(test)